Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Onnx Pipeline

This repository shows how to deploy and use Onnx pipeline with dockers including convert model, generate input and performance test.

# Prerequisites

Pull dockers from Azure. It should take several minutes.

In [5]:
!sh build.sh

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
Error response from daemon: Get https://ziylregistry.azurecr.io/v2/: unauthorized: authentication required
Using default tag: latest
latest: Pulling from onnx-converter
Digest: sha256:ab0f7a25d6c4afe0d1c33d43246af53ad63d2321804d37677e1187a257a2d4d9
Status: Image is up to date for ziylregistry.azurecr.io/onnx-converter:latest
Using default tag: latest
latest: Pulling from perf_test
Digest: sha256:0b93e6a1d4e4cd5e0057cf503fce53b8702d2445252b7837e844e50752d2a369
Status: Image is up to date for ziylregistry.azurecr.io/perf_test:latest


Install the onnxpipeline SDK

In [6]:
import onnxpipeline

# test tensorflow
#pipeline = onnxpipeline.Onnxpip('mnist/model')

# test pytorch
#pipeline = onnxpipeline.Onnxpip('pytorch')

# test mxnet fail
#pipeline = onnxpipeline.Onnxpip('mxnet')

# cntk 
#pipeline = onnxpipeline.Onnxpip('cntk')

# keras
pipeline = onnxpipeline.Onnxpip('KerasToONNX')

# caffe fail
#pipeline = onnxpipeline.Onnxpip('caffe')

# sklearn fail
#pipeline = onnxpipeline.Onnxpip('sklearn')

## Run parameters

(1) local_directory: string

    Required. The path of local directory where would be mounted to the docker. All operations will be executed from this path.

(2) mount_path: string

    Optional. The path where the local_directory will be mounted in the docker. Default is "/mnt/model".

(3) print_logs: boolean

    Optional. Whether print the logs from the docker. Default is True.


# Convert model to ONNX

This image is used to convert model from major model frameworks to onnx. Supported frameworks are - caffe, cntk, coreml, keras, libsvm, mxnet, scikit-learn, tensorflow and pytorch.


You can run the docker image with customized parameters.

In [7]:
# test tensorflow
#model = pipeline.convert_model(model_type='tensorflow')

# test pytorch
#model = pipeline.convert_model(model_type='pytorch', model='saved_model.pb', model_input_shapes='(1,3,224,224)')

# test mxnet
#model = pipeline.convert_model(model_type='mxnet', model='resnet.json', model_params='resnet.params', model_input_shapes='(1,3,224,224)')

# test cntk
#model = pipeline.convert_model(model_type='cntk', model='ResNet50_ImageNet_Caffe.model')

# test keras
model = pipeline.convert_model(model_type='keras', model='keras_Average_ImageNet.keras')

# test caffe
#model = pipeline.convert_model(model_type='caffe', model='generic_dnn_prod.caffemodel')

# test sklearn
#model = pipeline.convert_model(model_type='scikit-learn', model='sklearn_svc_prob.joblib')

2019-06-10 21:51:43.309018: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA

/usr/local/lib/python3.6/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.

  warnings.warn('No training configuration found in save file: '



-------------

Model Conversion



using tensorflow=1.12.0, onnx=1.5.0, opset=7, tfonnx=1.5.0/82f805



-------------

MODEL INPUT GENERATION(if needed)



Input.pb already exists. Skipping dummy input generation. 



-------------

MODEL CORRECTNESS VERIFICATION





Check the ONNX model for validity 

The ONNX model is valid.



Check ONNX model for correctness. 

No correctness verification method for keras model type



## Run parameters

(1) model: string

    Required. The path of the model that needs to be converted.

(2) output_onnx_path: string

    Required. The path to store the converted onnx model. Should end with ".onnx". e.g. output.onnx

(3) model_type: string

    Required. The name of original model framework. Available types are caffe, cntk, coreml, keras, libsvm, mxnet, scikit-learn, tensorflow and pytorch.

(4) model_inputs: string

    Optional. The model's input names. Required for tensorflow frozen models and checkpoints.

(5) model_outputs: string

    Optional. The model's output names. Required for tensorflow frozen models checkpoints.

(6) model_params: string

    Optional. The params of the model if needed.

(7) model_input_shapes: list of tuple

    Optional. List of tuples. The input shape(s) of the model. Each dimension separated by ','.

(8) target_opset: int

    Optional. Specifies the opset for ONNX, for example, 7 for ONNX 1.2, and 8 for ONNX 1.3. Defaults to 7.


# Performance test tool

You can run perf_test using command python perf_test.py [Your model path] [Output path on the docker]. You can use the same arguments as for onnxruntime_pert_test tool, e.g. -m for mode, -e to specify execution provider etc. By default it will try all providers available.

In [10]:
result = pipeline.perf_test(model=model, result="output.txt")
# result = pipeline.perf_test()   # is ok, too

Cores:  6



/home/artr/repo/onnxruntime/include/onnxruntime/core/session/onnxruntime_cxx_api.h:127 OrtCreateSession /home/artr/repo/onnxruntime/onnxruntime/core/providers/cuda/cuda_call.cc:97 bool onnxruntime::CudaCall(ERRTYPE, const char*, const char*, ERRTYPE, const char*) [with ERRTYPE = cudaError; bool THRW = true] /home/artr/repo/onnxruntime/onnxruntime/core/providers/cuda/cuda_call.cc:91 bool onnxruntime::CudaCall(ERRTYPE, const char*, const char*, ERRTYPE, const char*) [with ERRTYPE = cudaError; bool THRW = true] CUDA failure 35: CUDA driver version is insufficient for CUDA runtime version ; GPU=32525 ; hostname=8e74cae6bf6a ; expr=cudaSetDevice(device_id_); 

Stacktrace:



Stacktrace:







Setting thread pool size to 0





/home/artr/repo/onnxruntime/include/onnxruntime/core/session/onnxruntime_cxx_api.h:127 OrtCreateSession /home/artr/repo/onnxruntime/onnxruntime/core/providers/cuda/cuda_call.cc:97 bool onnxruntime::CudaCall(ERRTYPE, const char*, const char*, ERRTYPE, con

mkldnn_openmp OMP_NUM_THREADS=6, active 0.220772





Setting thread pool size to 0

Total time cost:0.00263009

Total iterations:10

Average time cost:0.263009 ms







Setting thread pool size to 0

Total time cost:0.00251563

Total iterations:10

Average time cost:0.251563 ms



mkldnn_openmp OMP_NUM_THREADS=5, passive 0.251563





Setting thread pool size to 0

Total time cost:0.00249772

Total iterations:10

Average time cost:0.249772 ms







Setting thread pool size to 0

Total time cost:0.0025512

Total iterations:10

Average time cost:0.25512 ms



mkldnn_openmp OMP_NUM_THREADS=5, active 0.25512





Setting thread pool size to 0

Total time cost:0.00285633

Total iterations:10

Average time cost:0.285633 ms







Setting thread pool size to 0

Total time cost:0.00287652

Total iterations:10

Average time cost:0.287652 ms



mkldnn_openmp OMP_NUM_THREADS=4, passive 0.287652





Setting thread pool size to 0

Total time cost:0.0023749

Total iterations:10

Average time cos

# Run parameters

(1) model: string

    Required. The path of the model that wants to be performed.
    
(2) result: string

    Required. The path of the result.
    

In [11]:
pipeline.print_result(result)
#pipeline.print_result() # is ok, too

ngraph 0.177807 ms

cpu_openmp OMP_NUM_THREADS=5, passive 0.205402 ms

cpu_openmp OMP_NUM_THREADS=5, active 0.209938 ms

mkldnn_openmp OMP_NUM_THREADS=4, active 0.217633 ms

mkldnn_openmp OMP_NUM_THREADS=6, active 0.220772 ms

cpu_openmp OMP_NUM_THREADS=6, passive 0.236715 ms

cpu_openmp OMP_NUM_THREADS=6, active 0.241648 ms

cpu_openmp active 0.250197 ms

mkldnn_openmp OMP_NUM_THREADS=5, passive 0.251563 ms

mkldnn_openmp OMP_NUM_THREADS=5, active 0.25512 ms

cpu_openmp passive 0.258029 ms

cpu_openmp OMP_NUM_THREADS=1, passive 0.264476 ms

mkldnn_openmp OMP_NUM_THREADS=3, passive 0.266843 ms

cpu_openmp OMP_NUM_THREADS=4, passive 0.277933 ms

mkldnn_openmp passive 0.278145 ms

cpu_openmp OMP_NUM_THREADS=2, active 0.286094 ms

mkldnn_openmp OMP_NUM_THREADS=1, active 0.286294 ms

mkldnn_openmp OMP_NUM_THREADS=3, active 0.286776 ms

mkldnn_openmp OMP_NUM_THREADS=4, passive 0.287652 ms

mkldnn_openmp OMP_NUM_THREADS=6, passive 0.290768 ms

cpu_openmp OMP_NUM_THREADS=4, active 0.298209 ms

# Run parameters

(1) result: string
Optional. The path of the result.

# netron

In [5]:
# still trying
import netron
netron.start('../deepthink.onnx')
from IPython.display import IFrame
IFrame('http://10.161.18.106:8080', width=700, height=350)

Serving '../deepthink.onnx' at http://localhost:8080
